In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("all sephora.csv", encoding="utf-8", index_col=0)

In [3]:
print(df.columns)

Index(['AuthorId', 'Helpfulness', 'Id', 'LastModificationTime', 'ProductId',
       'Rating', 'RatingRange', 'ReviewText', 'UserNickname', 'age',
       'eyeColor', 'hairColor', 'skinTone', 'skinType'],
      dtype='object')


In [4]:
columns = ["eyeColor", "hairColor", "skinTone", "skinType"]

In [5]:
skus = set(df["ProductId"])

In [6]:
# skus

In [16]:
label_dict = {}

for col in columns:
    label_dict[col] = set(df[col])

import pprint
pprint.pprint(label_dict)

{'eyeColor': {nan, 'green', 'blue', 'hazel', 'gray', 'brown'},
 'hairColor': {nan, 'brunette', 'blonde', 'red', 'black', 'auburn', 'gray'},
 'skinTone': {nan,
              'dark',
              'deep',
              'ebony',
              'fair',
              'light',
              'medium',
              'olive',
              'porcelain',
              'tan'},
 'skinType': {nan, 'oily', 'normal', 'dry', 'combination'}}


In [17]:
for col in label_dict["eyeColor"]:
    if pd.isnull(col):
        print("NAN", type(col))
    else:
        print(col)

NAN <class 'float'>
green
blue
hazel
gray
brown


In [9]:
df[pd.isnull(df["skinType"]) | (df["skinType"] == "dry")].head(50)

,AuthorId,Helpfulness,Id,LastModificationTime,ProductId,Rating,RatingRange,ReviewText,UserNickname,age,eyeColor,hairColor,skinTone,skinType
8,1502678847,1.000000,124469579,2019-04-08T05:42:12.000+00:00,2144608,5,5,"RiRi did it again!\r\nI LOVE this set, from th...",purplemoon,13to17,brown,brunette,fair,dry
9,11745190902,NaN,124433325,2019-02-12T17:00:07.000+00:00,2144608,5,5,Love love love!! They definitely hit it out of...,thewhitejag,NaN,blue,brunette,porcelain,dry
24,1650932125,NaN,123165749,2019-01-09T13:45:04.000+00:00,2144608,5,5,I bought this for the hot pink and it is even ...,KRReed,NaN,brown,brunette,medium,dry
25,1650932125,NaN,123165712,2019-01-09T13:45:04.000+00:00,2144616,5,5,Love the colors! The blue isn't too overpoweri...,KRReed,NaN,brown,brunette,medium,dry
27,5213721427,NaN,123138536,2019-01-08T05:45:03.000+00:00,2144616,5,5,This is so out of my usual color range but I a...,MidnightDreamer,NaN,brown,brunette,fair,dry
28,2090226998,NaN,123053679,2019-01-05T02:30:04.000+00:00,2144616,4,5,"Pretty colours, love the packaging. Glides on...",2manyperfumes,NaN,hazel,brunette,fair,dry
44,2170902292,NaN,121364514,2018-11-14T05:15:02.000+00:00,2144616,5,5,Love the colors. Comfortable to wear & gorgeou...,lecaptain,NaN,brown,blonde,porcelain,dry
47,6854553786,1.000000,121255643,2019-03-28T02:08:55.000+00:00,2144608,5,5,"Fenty's ""Snow Daze"" is absolute perfection! F...",GianettaB,NaN,green,brunette,light,dry
54,2077014259,1.000000,120829387,2019-03-29T21:43:21.000+00:00,2144608,5,5,"More moisturizing than mattemoisel, very beaut...",AndiSilva,NaN,brown,brunette,tan,dry
62,22881223329,NaN,126274976,2019-04-07T23:15:04.000+00:00,2063766,1,5,"Don’t do it!! I love Nars orgasm everything, b...",Fancy001,NaN,blue,black,olive,NaN


In [20]:
df["Rating"].dtype

dtype('int64')

In [32]:
len(skus)

3576

In [35]:
import time

total = len(skus)

result = {}

t0 = time.time()

counter = 0

for i, sku in enumerate(skus):
    buf = df[df["ProductId"] == sku]
    scores = {}
    for j, col in enumerate(columns):
        if pd.isnull(col):
            continue
        labels = label_dict[col]
        scores[col] = {}
        for k, label in enumerate(labels):
            filtered = buf[pd.isnull(buf[col]) | (buf[col] == label)]
            counter += len(filtered)
            mean = filtered["Rating"].mean()
            scores[col][label] = mean
    result[sku] = scores
    if i % 100 == 0:
        dt = time.time() - t0
        print("{:>4d}: counter = {:>6d}, used = {:.1f}, eta = {:.1f}".format(i, counter, dt, dt * (total - i - 1) / (i + 1)))

   0: counter =     33, used = 0.0, eta = 139.3
 100: counter = 131472, used = 3.1, eta = 105.8
 200: counter = 320320, used = 6.1, eta = 102.7
 300: counter = 348506, used = 9.1, eta = 99.3
 400: counter = 421967, used = 12.2, eta = 96.4
 500: counter = 493597, used = 15.2, eta = 93.2
 600: counter = 515194, used = 18.2, eta = 90.1
 700: counter = 539036, used = 21.2, eta = 86.9
 800: counter = 602541, used = 24.3, eta = 84.0
 900: counter = 671086, used = 27.3, eta = 81.1
1000: counter = 695954, used = 30.4, eta = 78.1
1100: counter = 750186, used = 33.4, eta = 75.2
1200: counter = 758775, used = 36.4, eta = 72.0
1300: counter = 772123, used = 39.4, eta = 68.9
1400: counter = 850069, used = 42.4, eta = 65.9
1500: counter = 999758, used = 45.5, eta = 62.9
1600: counter = 1041814, used = 48.5, eta = 59.8
1700: counter = 1067715, used = 51.5, eta = 56.8
1800: counter = 1381579, used = 54.6, eta = 53.8
1900: counter = 1413868, used = 57.6, eta = 50.8
2000: counter = 1492138, used = 60.6,

In [38]:
counter, len(df)

(2290924, 157456)

In [42]:
import copy

raw_result = copy.deepcopy(result)

In [43]:
raw_result

{'1944826': {'eyeColor': {nan: 5.0,
   'green': 5.0,
   'blue': 4.0,
   'hazel': 5.0,
   'gray': 5.0,
   'brown': 4.0},
  'hairColor': {nan: nan,
   'brunette': 3.5,
   'blonde': 4.0,
   'red': nan,
   'black': 5.0,
   'auburn': nan,
   'gray': nan},
  'skinTone': {'deep': nan,
   nan: nan,
   'ebony': nan,
   'medium': 1.0,
   'fair': 4.0,
   'light': 5.0,
   'tan': 5.0,
   'olive': 3.0,
   'dark': nan,
   'porcelain': 5.0},
  'skinType': {nan: nan,
   'oily': 3.0,
   'normal': 4.0,
   'dry': 5.0,
   'combination': 1.0}},
 '1674829': {'eyeColor': {nan: nan,
   'green': nan,
   'blue': 4.0,
   'hazel': nan,
   'gray': nan,
   'brown': nan},
  'hairColor': {nan: nan,
   'brunette': 4.0,
   'blonde': nan,
   'red': nan,
   'black': nan,
   'auburn': nan,
   'gray': nan},
  'skinTone': {'deep': nan,
   nan: nan,
   'ebony': nan,
   'medium': nan,
   'fair': 4.0,
   'light': nan,
   'tan': nan,
   'olive': nan,
   'dark': nan,
   'porcelain': nan},
  'skinType': {nan: nan,
   'oily': 4.0,


In [44]:
for sku, sku_scores in result.items():
    for col, col_scores in sku_scores.items():
        for label, score in col_scores.items():
            if np.isnan(score):
                result[sku][col][label] = df[df["ProductId"] == sku]["Rating"].mean()

In [45]:
result

{'1944826': {'eyeColor': {nan: 5.0,
   'green': 5.0,
   'blue': 4.0,
   'hazel': 5.0,
   'gray': 5.0,
   'brown': 4.0},
  'hairColor': {nan: 3.857142857142857,
   'brunette': 3.5,
   'blonde': 4.0,
   'red': 3.857142857142857,
   'black': 5.0,
   'auburn': 3.857142857142857,
   'gray': 3.857142857142857},
  'skinTone': {'deep': 3.857142857142857,
   nan: 3.857142857142857,
   'ebony': 3.857142857142857,
   'medium': 1.0,
   'fair': 4.0,
   'light': 5.0,
   'tan': 5.0,
   'olive': 3.0,
   'dark': 3.857142857142857,
   'porcelain': 5.0},
  'skinType': {nan: 3.857142857142857,
   'oily': 3.0,
   'normal': 4.0,
   'dry': 5.0,
   'combination': 1.0}},
 '1674829': {'eyeColor': {nan: 4.0,
   'green': 4.0,
   'blue': 4.0,
   'hazel': 4.0,
   'gray': 4.0,
   'brown': 4.0},
  'hairColor': {nan: 4.0,
   'brunette': 4.0,
   'blonde': 4.0,
   'red': 4.0,
   'black': 4.0,
   'auburn': 4.0,
   'gray': 4.0},
  'skinTone': {'deep': 4.0,
   nan: 4.0,
   'ebony': 4.0,
   'medium': 4.0,
   'fair': 4.0,
  

In [46]:
import json

with open("label rating.json", "w", encoding="utf-8") as fout:
    json.dump(result, fout)